<a href="https://colab.research.google.com/github/sean-halpin/diffusion_models/blob/main/min_diffusion_huggingface_unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf /content/images/*

In [ ]:
!mkdir -p /content/images/

In [ ]:
from typing import Dict, Tuple
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# from torchvision.datasets import MNIST
from torchvision import transforms
from torchvision.utils import save_image, make_grid

In [ ]:
!python -m pip uninstall matplotlib -y
!pip install matplotlib==3.1.3

### UNet

#### UNET 1

In [ ]:
# """
# Simple Unet Structure.
# """
# import torch
# import torch.nn as nn
# import torch.nn.functional as F


# class Conv3(nn.Module):
#     def __init__(
#         self, in_channels: int, out_channels: int, is_res: bool = False
#     ) -> None:
#         super().__init__()
#         self.main = nn.Sequential(
#             nn.Conv2d(in_channels, out_channels, 3, 1, 1),
#             nn.GroupNorm(8, out_channels),
#             nn.ReLU(),
#         )
#         self.conv = nn.Sequential(
#             nn.Conv2d(out_channels, out_channels, 3, 1, 1),
#             nn.GroupNorm(8, out_channels),
#             nn.ReLU(),
#             nn.Conv2d(out_channels, out_channels, 3, 1, 1),
#             nn.GroupNorm(8, out_channels),
#             nn.ReLU(),
#         )

#         self.is_res = is_res

#     def forward(self, x: torch.Tensor) -> torch.Tensor:
#         x = self.main(x)
#         if self.is_res:
#             x = x + self.conv(x)
#             return x / 1.414
#         else:
#             return self.conv(x)


# class UnetDown(nn.Module):
#     def __init__(self, in_channels: int, out_channels: int) -> None:
#         super(UnetDown, self).__init__()
#         layers =  [
#                     Conv3(in_channels, out_channels), nn.MaxPool2d(2)
#                   ]
#         self.model = nn.Sequential(*layers)

#     def forward(self, x: torch.Tensor) -> torch.Tensor:

#         return self.model(x)


# class UnetUp(nn.Module):
#     def __init__(self, in_channels: int, out_channels: int) -> None:
#         super(UnetUp, self).__init__()
#         layers = [
#             nn.ConvTranspose2d(in_channels, out_channels, 2, 2),
#             Conv3(out_channels, out_channels),
#             Conv3(out_channels, out_channels),
#         ]
#         self.model = nn.Sequential(*layers)

#     def forward(self, x: torch.Tensor, skip: torch.Tensor) -> torch.Tensor:
#         x = torch.cat((x, skip), 1)
#         x = self.model(x)

#         return x


# class TimeSiren(nn.Module):
#     def __init__(self, emb_dim: int) -> None:
#         super(TimeSiren, self).__init__()

#         self.lin1 = nn.Linear(1, emb_dim, bias=False)
#         self.lin2 = nn.Linear(emb_dim, emb_dim)

#     def forward(self, x: torch.Tensor) -> torch.Tensor:
#         x = x.view(-1, 1)
#         x = torch.sin(self.lin1(x))
#         x = self.lin2(x)
#         return x


# class NaiveUnet(nn.Module):
#     def __init__(self, in_channels: int, out_channels: int, n_feat: int = 256, attribute_count=40) -> None:
#         super(NaiveUnet, self).__init__()
#         self.in_channels = in_channels
#         self.out_channels = out_channels

#         self.n_feat = n_feat

#         self.init_conv = Conv3(in_channels, n_feat, is_res=True)

#         self.labels = nn.Sequential(
#             nn.Conv2d(attribute_count, n_feat, 3,2,32),
#             nn.GroupNorm(8, n_feat),
#             nn.ReLU()
#         )
#         n_feat = n_feat*2
#         self.down1 = UnetDown(n_feat, n_feat)
#         self.down2 = UnetDown(n_feat, 2 * n_feat)
#         self.down3 = UnetDown(2 * n_feat, 2 * n_feat)

#         self.to_vec = nn.Sequential(nn.AvgPool2d(4), nn.ReLU())

#         self.timeembed = TimeSiren(2 * n_feat)

#         self.up0 = nn.Sequential(
#             nn.ConvTranspose2d(2 * n_feat, 2 * n_feat, 4, 4),
#             nn.GroupNorm(8, 2 * n_feat),
#             nn.ReLU(),
#         )

#         self.up1 = UnetUp(4 * n_feat, 2 * n_feat)
#         self.up2 = UnetUp(4 * n_feat, n_feat)
#         self.up3 = UnetUp(2 * n_feat, n_feat)
#         self.out = nn.Conv2d(192, self.out_channels, 3, 1, 1)
#         # self.out = nn.Conv2d(2 * n_feat, self.out_channels, 3, 1, 1)

#     def forward(self, x: torch.Tensor, t: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
#         # print("X DEBUG")
#         # print(x.shape)
#         xx = self.init_conv(x)
#         # x = self.init_conv(x)
#         # print(xx.shape)
#         # print("Y DEBUG")
#         # print(y.shape)
#         yy = y.view(y.shape[0], y.shape[1], 1,1)
#         # print(yy.shape)
#         yyy = self.labels(yy)
#         # print(yyy.shape)
#         # print("END DEBUG")
#         xy = torch.cat([xx,yyy], dim=1)

#         down1 = self.down1(xy)
#         # down1 = self.down1(x)
#         down2 = self.down2(down1)
#         down3 = self.down3(down2)

#         thro = self.to_vec(down3)
#         temb = self.timeembed(t).view(-1, self.n_feat * 4, 1, 1)
#         # temb = self.timeembed(t).view(-1, self.n_feat * 2, 1, 1)

#         thro = self.up0(thro + temb)

#         up1 = self.up1(thro, down3) + temb
#         up2 = self.up2(up1, down2)
#         up3 = self.up3(up2, down1)
#         # print("OUT DEBUG")
#         # print(up3.shape)
#         # print(xx.shape)
#         # print(x.shape)
#         # print("END DEBUG")
#         out = self.out(torch.cat((up3, xx), 1))

#         return out

#### UNET 2

In [ ]:
# !pip install -q -U einops datasets matplotlib tqdm

# import math
# from inspect import isfunction
# from functools import partial

# %matplotlib inline
# import matplotlib.pyplot as plt
# from tqdm.auto import tqdm
# from einops import rearrange

# import torch
# from torch import nn, einsum
# import torch.nn.functional as F

In [ ]:
# def exists(x):
#     return x is not None

# def default(val, d):
#     if exists(val):
#         return val
#     return d() if isfunction(d) else d

# class Residual(nn.Module):
#     def __init__(self, fn):
#         super().__init__()
#         self.fn = fn

#     def forward(self, x, *args, **kwargs):
#         return self.fn(x, *args, **kwargs) + x

# def Upsample(dim):
#     return nn.ConvTranspose2d(dim, dim, 4, 2, 1)

# def Downsample(dim):
#     return nn.Conv2d(dim, dim, 4, 2, 1)

In [ ]:
# class SinusoidalPositionEmbeddings(nn.Module):
#     def __init__(self, dim):
#         super().__init__()
#         self.dim = dim

#     def forward(self, time):
#         device = time.device
#         half_dim = self.dim // 2
#         embeddings = math.log(10000) / (half_dim - 1)
#         embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
#         embeddings = time[:, None] * embeddings[None, :]
#         embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
#         return embeddings

In [ ]:
# class Block(nn.Module):
#     def __init__(self, dim, dim_out, groups = 8):
#         super().__init__()
#         self.proj = nn.Conv2d(dim, dim_out, 3, padding = 1)
#         self.norm = nn.GroupNorm(groups, dim_out)
#         self.act = nn.SiLU()

#     def forward(self, x, scale_shift = None):
#         x = self.proj(x)
#         x = self.norm(x)

#         if exists(scale_shift):
#             scale, shift = scale_shift
#             x = x * (scale + 1) + shift

#         x = self.act(x)
#         return x

# class ResnetBlock(nn.Module):
#     """https://arxiv.org/abs/1512.03385"""
    
#     def __init__(self, dim, dim_out, *, time_emb_dim=None, groups=8):
#         super().__init__()
#         self.mlp = (
#             nn.Sequential(nn.SiLU(), nn.Linear(time_emb_dim, dim_out))
#             if exists(time_emb_dim)
#             else None
#         )

#         self.block1 = Block(dim, dim_out, groups=groups)
#         self.block2 = Block(dim_out, dim_out, groups=groups)
#         self.res_conv = nn.Conv2d(dim, dim_out, 1) if dim != dim_out else nn.Identity()

#     def forward(self, x, time_emb=None):
#         h = self.block1(x)

#         if exists(self.mlp) and exists(time_emb):
#             time_emb = self.mlp(time_emb)
#             # h = rearrange(time_emb, "b z c -> b c z 1") + h
#             h = rearrange(time_emb, "b c -> b c 1 1") + h

#         h = self.block2(h)
#         return h + self.res_conv(x)
    
# class ConvNextBlock(nn.Module):
#     """https://arxiv.org/abs/2201.03545"""

#     def __init__(self, dim, dim_out, *, time_emb_dim=None, mult=2, norm=True):
#         super().__init__()
#         self.mlp = (
#             nn.Sequential(nn.GELU(), nn.Linear(time_emb_dim, dim))
#             if exists(time_emb_dim)
#             else None
#         )

#         self.ds_conv = nn.Conv2d(dim, dim, 7, padding=3, groups=dim)

#         self.net = nn.Sequential(
#             nn.GroupNorm(1, dim) if norm else nn.Identity(),
#             nn.Conv2d(dim, dim_out * mult, 3, padding=1),
#             nn.GELU(),
#             nn.GroupNorm(1, dim_out * mult),
#             nn.Conv2d(dim_out * mult, dim_out, 3, padding=1),
#         )

#         self.res_conv = nn.Conv2d(dim, dim_out, 1) if dim != dim_out else nn.Identity()

#     def forward(self, x, time_emb=None):
#         h = self.ds_conv(x)

#         if exists(self.mlp) and exists(time_emb):
#             assert exists(time_emb), "time embedding must be passed in"
#             condition = self.mlp(time_emb)
#             if condition.ndim == 2:
#               h = h + rearrange(condition, "b c -> b c 1 1")
#             else:
#               h = h + rearrange(condition, "b z c -> b c z 1")

#         h = self.net(h)
#         return h + self.res_conv(x)

In [ ]:
# class Attention(nn.Module):
#     def __init__(self, dim, heads=4, dim_head=32):
#         super().__init__()
#         self.scale = dim_head**-0.5
#         self.heads = heads
#         hidden_dim = dim_head * heads
#         self.to_qkv = nn.Conv2d(dim, hidden_dim * 3, 1, bias=False)
#         self.to_out = nn.Conv2d(hidden_dim, dim, 1)

#     def forward(self, x):
#         b, c, h, w = x.shape
#         qkv = self.to_qkv(x).chunk(3, dim=1)
#         q, k, v = map(
#             lambda t: rearrange(t, "b (h c) x y -> b h c (x y)", h=self.heads), qkv
#         )
#         q = q * self.scale

#         sim = einsum("b h d i, b h d j -> b h i j", q, k)
#         sim = sim - sim.amax(dim=-1, keepdim=True).detach()
#         attn = sim.softmax(dim=-1)

#         out = einsum("b h i j, b h d j -> b h i d", attn, v)
#         out = rearrange(out, "b h (x y) d -> b (h d) x y", x=h, y=w)
#         return self.to_out(out)

# class LinearAttention(nn.Module):
#     def __init__(self, dim, heads=4, dim_head=32):
#         super().__init__()
#         self.scale = dim_head**-0.5
#         self.heads = heads
#         hidden_dim = dim_head * heads
#         self.to_qkv = nn.Conv2d(dim, hidden_dim * 3, 1, bias=False)

#         self.to_out = nn.Sequential(nn.Conv2d(hidden_dim, dim, 1), 
#                                     nn.GroupNorm(1, dim))

#     def forward(self, x):
#         b, c, h, w = x.shape
#         qkv = self.to_qkv(x).chunk(3, dim=1)
#         q, k, v = map(
#             lambda t: rearrange(t, "b (h c) x y -> b h c (x y)", h=self.heads), qkv
#         )

#         q = q.softmax(dim=-2)
#         k = k.softmax(dim=-1)

#         q = q * self.scale
#         context = torch.einsum("b h d n, b h e n -> b h d e", k, v)

#         out = torch.einsum("b h d e, b h d n -> b h e n", context, q)
#         out = rearrange(out, "b h c (x y) -> b (h c) x y", h=self.heads, x=h, y=w)
#         return self.to_out(out)

In [ ]:
# class PreNorm(nn.Module):
#     def __init__(self, dim, fn):
#         super().__init__()
#         self.fn = fn
#         self.norm = nn.GroupNorm(1, dim)

#     def forward(self, x):
#         x = self.norm(x)
#         return self.fn(x)

In [ ]:
# # https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/annotated_diffusion.ipynb?authuser=1#scrollTo=a5126e21
# class Unet(nn.Module):
#     def __init__(
#         self,
#         dim,
#         init_dim=None,
#         out_dim=None,
#         dim_mults=(1, 2, 4, 8),
#         channels=3,
#         with_time_emb=True,
#         resnet_block_groups=8,
#         use_convnext=True,
#         convnext_mult=2,
#     ):
#         super().__init__()

#         # determine dimensions
#         self.channels = channels

#         init_dim = default(init_dim, dim // 3 * 2)
#         self.init_conv = nn.Conv2d(channels, init_dim, 7, padding=3)

#         dims = [init_dim, *map(lambda m: dim * m, dim_mults)]
#         in_out = list(zip(dims[:-1], dims[1:]))
        
#         if use_convnext:
#             block_klass = partial(ConvNextBlock, mult=convnext_mult)
#         else:
#             block_klass = partial(ResnetBlock, groups=resnet_block_groups)

#         # time embeddings
#         if with_time_emb:
#             time_dim = dim * 4
#             self.time_mlp = nn.Sequential(
#                 SinusoidalPositionEmbeddings(dim),
#                 nn.Linear(dim, time_dim),
#                 nn.GELU(),
#                 nn.Linear(time_dim, time_dim),
#             )
#         else:
#             time_dim = None
#             self.time_mlp = None

#         # layers
#         self.downs = nn.ModuleList([])
#         self.ups = nn.ModuleList([])
#         num_resolutions = len(in_out)

#         for ind, (dim_in, dim_out) in enumerate(in_out):
#             is_last = ind >= (num_resolutions - 1)

#             self.downs.append(
#                 nn.ModuleList(
#                     [
#                         block_klass(dim_in, dim_out, time_emb_dim=time_dim),
#                         block_klass(dim_out, dim_out, time_emb_dim=time_dim),
#                         Residual(PreNorm(dim_out, LinearAttention(dim_out))),
#                         Downsample(dim_out) if not is_last else nn.Identity(),
#                     ]
#                 )
#             )

#         mid_dim = dims[-1]
#         self.mid_block1 = block_klass(mid_dim, mid_dim, time_emb_dim=time_dim)
#         self.mid_attn = Residual(PreNorm(mid_dim, Attention(mid_dim)))
#         self.mid_block2 = block_klass(mid_dim, mid_dim, time_emb_dim=time_dim)

#         for ind, (dim_in, dim_out) in enumerate(reversed(in_out[1:])):
#             is_last = ind >= (num_resolutions - 1)

#             self.ups.append(
#                 nn.ModuleList(
#                     [
#                         block_klass(dim_out * 2, dim_in, time_emb_dim=time_dim),
#                         block_klass(dim_in, dim_in, time_emb_dim=time_dim),
#                         Residual(PreNorm(dim_in, LinearAttention(dim_in))),
#                         Upsample(dim_in) if not is_last else nn.Identity(),
#                     ]
#                 )
#             )

#         out_dim = default(out_dim, channels)
#         self.final_conv = nn.Sequential(
#             block_klass(dim, dim), nn.Conv2d(dim, out_dim, 1)
#         )

#     def forward(self, x, time):
#         x = self.init_conv(x)

#         t = self.time_mlp(time) if exists(self.time_mlp) else None

#         h = []

#         # downsample
#         for block1, block2, attn, downsample in self.downs:
#             x = block1(x, t)
#             x = block2(x, t)
#             x = attn(x)
#             h.append(x)
#             x = downsample(x)

#         # bottleneck
#         x = self.mid_block1(x, t)
#         x = self.mid_attn(x)
#         x = self.mid_block2(x, t)

#         # upsample
#         for block1, block2, attn, upsample in self.ups:
#             x = torch.cat((x, h.pop()), dim=1)
#             x = block1(x, t)
#             x = block2(x, t)
#             x = attn(x)
#             x = upsample(x)

#         return self.final_conv(x)

#### UNET 4

In [ ]:
!pip install diffusers 

In [ ]:
from diffusers import UNet2DModel

### Denoising Diffusion Probabilistic Model

In [ ]:
from typing import Dict, Tuple


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from typing import Dict, Optional, Tuple


class DDPM(nn.Module):
    def __init__(
        self,
        eps_model: nn.Module,
        betas: Tuple[float, float],
        n_T: int,
        criterion: nn.Module = nn.MSELoss(),
    ) -> None:
        super(DDPM, self).__init__()
        self.eps_model = eps_model

        # register_buffer allows us to freely access these tensors by name. It helps device placement.
        for k, v in ddpm_schedules(betas[0], betas[1], n_T).items():
            self.register_buffer(k, v)

        self.n_T = n_T
        self.criterion = criterion

    def forward(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        """
        Makes forward diffusion x_t, and tries to guess epsilon value from x_t using eps_model.
        This implements Algorithm 1 in the paper.
        """

        _ts = torch.randint(1, self.n_T + 1, (x.shape[0],)).to(x.device)
        # t ~ Uniform(0, n_T)
        eps = torch.randn_like(x)  # eps ~ N(0, 1)

        x_t = (
            self.sqrtab[_ts, None, None, None] * x
            + self.sqrtmab[_ts, None, None, None] * eps
        )  # This is the x_t, which is sqrt(alphabar) x_0 + sqrt(1-alphabar) * eps
        # We should predict the "error term" from this x_t. Loss is what we return.

        res = self.eps_model(x_t, _ts / self.n_T)
        # print(res.keys())

        return self.criterion(eps, res['sample']) #, y=y))

    def sample(self, n_sample: int, size, device, attrs = None) -> torch.Tensor:

        # x_i = torch.randn(n_sample, *size).to(device)  # x_T ~ N(0, 1)
        x_i = torch.randn(n_sample, *size).to(device)


        # This samples accordingly to Algorithm 2. It is exactly the same logic.
        for i in range(self.n_T, 0, -1):
            z = torch.randn(n_sample, *size).to(device) if i > 1 else 0
            eps = self.eps_model(
                x_i, torch.tensor(i / self.n_T).to(device).repeat(n_sample, 1)#, attrs
            )
            x_i = (
                self.oneover_sqrta[i] * (x_i - eps * self.mab_over_sqrtmab[i])
                + self.sqrt_beta_t[i] * z
            )

        return x_i


def ddpm_schedules(beta1: float, beta2: float, T: int) -> Dict[str, torch.Tensor]:
    """
    Returns pre-computed schedules for DDPM sampling, training process.
    """
    assert beta1 < beta2 < 1.0, "beta1 and beta2 must be in (0, 1)"

    beta_t = (beta2 - beta1) * torch.arange(0, T + 1, dtype=torch.float32) / T + beta1
    sqrt_beta_t = torch.sqrt(beta_t)
    alpha_t = 1 - beta_t
    log_alpha_t = torch.log(alpha_t)
    alphabar_t = torch.cumsum(log_alpha_t, dim=0).exp()

    sqrtab = torch.sqrt(alphabar_t)
    oneover_sqrta = 1 / torch.sqrt(alpha_t)

    sqrtmab = torch.sqrt(1 - alphabar_t)
    mab_over_sqrtmab_inv = (1 - alpha_t) / sqrtmab

    return {
        "alpha_t": alpha_t,  # \alpha_t
        "oneover_sqrta": oneover_sqrta,  # 1/\sqrt{\alpha_t}
        "sqrt_beta_t": sqrt_beta_t,  # \sqrt{\beta_t}
        "alphabar_t": alphabar_t,  # \bar{\alpha_t}
        "sqrtab": sqrtab,  # \sqrt{\bar{\alpha_t}}
        "sqrtmab": sqrtmab,  # \sqrt{1-\bar{\alpha_t}}
        "mab_over_sqrtmab": mab_over_sqrtmab_inv,  # (1-\alpha_t)/\sqrt{1-\bar{\alpha_t}}
    }

### Dataset

In [ ]:
# from torchvision.datasets import CIFAR10
from torchvision.datasets import CelebA

In [ ]:
x_size = 32
y_size = 32

In [ ]:
tf = transforms.Compose(
  [
    transforms.Resize((x_size, y_size)),
    transforms.ToTensor(), 
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ]
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!mkdir -p ./data
!rm -rf ./data/*

In [ ]:
!cp -R ./drive/MyDrive/celeba/* ./data/celeba

In [ ]:
!unzip "./data/celeba/img_align_celeba.zip" -d "./data/celeba"

In [ ]:
import time 

dataset = None
result = None
while result is None:
    try:
      # connect
      dataset = CelebA(
        "./data/",
        download=False,
        transform=tf,
      )
      result = True
    except Exception as e:
      print(e)
      # break
      time.sleep(60)
      pass


In [ ]:
subset = torch.utils.data.Subset(dataset, range(0,2560))

In [ ]:
# dataloader = DataLoader(subset, batch_size=512, shuffle=True, num_workers=2)
dataloader = DataLoader(dataset, batch_size=512, shuffle=True, num_workers=2)

### Train DDPM

In [ ]:
model = UNet2DModel(
        sample_size=32,
        in_channels=3,
        out_channels=3,
        layers_per_block=2,
        block_out_channels=(32, 32, 64), #, 256), #, 512, 512),
        # block_out_channels=(128, 128, 256, 256, 512, 512),
        down_block_types=(
            # "DownBlock2D",
            # "DownBlock2D",
            # "DownBlock2D",
            "DownBlock2D",
            "AttnDownBlock2D",
            "DownBlock2D",
        ),
        up_block_types=(
            "UpBlock2D",
            "AttnUpBlock2D",
            "UpBlock2D",
            # "UpBlock2D",
            # "UpBlock2D",
            # "UpBlock2D",
        ),
    )

In [ ]:
import matplotlib.pyplot as plt
from typing import Dict, Optional, Tuple
from sympy import Ci
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.utils import save_image, make_grid

from pathlib import Path


def create_model(load_pth = False, device: str = "cuda:0"):
    # ddpm = DDPM(eps_model=NaiveUnet(3, 3, n_feat=64, attribute_count = dataset.attr.shape[1]), 
    #             betas=(1e-4, 0.02), n_T=1000,
    #             criterion=nn.MSELoss())
    # ddpm = DDPM(eps_model=Unet(dim=x_size,channels=3,dim_mults=(1, 2, 4, 8)), 
    #         betas=((1e-4)*20, (0.02)*20), n_T=1000,
    #         criterion=nn.MSELoss())
    ddpm = DDPM(eps_model=model, 
            betas=((1e-4), (0.02)), n_T=1000,
            criterion=nn.MSELoss())
    if load_pth:
      my_file = Path("ddpm.pth")
      if my_file.is_file(): 
        print("Loading Model from File")
        ddpm.load_state_dict(torch.load("ddpm.pth"))
      else:
        my_file = Path("./drive/MyDrive/thesis_models/ddpm.pth")
        if my_file.is_file(): 
          print("Loading Model from File")
          ddpm.load_state_dict(torch.load("./drive/MyDrive/thesis_models/ddpm.pth"))
    ddpm.to(device)
    return ddpm

def train(ddpm: Optional[any] = None, n_epoch: int = 1, device: str = "cuda:0", 
          dataloader = None, x_size=32, y_size=32, modulo_print=1) -> None:

    if ddpm == None:
      print("Creating Model")
      ddpm = create_model()

    optim = torch.optim.Adam(ddpm.parameters(), lr=1e-5)

    for i in range(n_epoch):
        print(f"Epoch {i} : ")
        ddpm.train()

        pbar = tqdm(dataloader)
        loss_ema = None
        for x, y in pbar:
            optim.zero_grad()
            x = x.to(device)
            y = y.to(device)
            # print(y)
            loss = ddpm(x,y.float())
            loss.backward()
            if loss_ema is None:
                loss_ema = loss.item()
            else:
                loss_ema = 0.9 * loss_ema + 0.1 * loss.item()
            pbar.set_description(f"loss: {loss_ema:.4f}")
            optim.step()

        ddpm.eval()
        with torch.no_grad():
          if i % modulo_print == 0:
            test_rand = (torch.rand(size=(8,dataset.attr.shape[1])) < 0.96).int().float().to(device)
            xh = ddpm.sample(8, (3, x_size, y_size), device, test_rand)
            xset = torch.cat([xh, x[:8]], dim=0)
            grid = make_grid(xset, normalize=True, value_range=(-1, 1), nrow=4)
            plt.imshow(
                transforms.functional.vflip(
                  transforms.functional.rotate(grid.cpu(), 90).T)
                )
            plt.show()
            save_image(grid, f"/content/images/ddpm_sample_{i}.png")
            # save model
            torch.save(ddpm.state_dict(), f"./ddpm.pth")
            torch.save(ddpm.state_dict(), f"./drive/MyDrive/thesis_models/ddpm.pth")

### Begin Training

In [ ]:
ddpm = create_model(load_pth=True)

In [ ]:
# del ddpm
import torch
import gc
gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()
# torch.cuda.memory_summary(device=None, abbreviated=False)
!nvidia-smi

In [ ]:
train(ddpm, n_epoch=100, dataloader=dataloader, x_size=x_size, y_size=y_size, modulo_print=10)

loss: 1.0287:  16%|█▌        | 50/318 [00:31<02:44,  1.63it/s]